# Problem Description

Consider the diophantine equation $\frac{1}{a}+\frac{1}{b}= \frac{p}{10^n}$ with $a, b, p, n$ positive integers and $a ≤ b$.

How many solutions has this equation for $1 ≤ n ≤ 9$?

Rearrange equation as follows:

$10^n(a + b) = pab$

For a given value of $a$ we can find the minimum value of $p$ as follows:

$p_{min} = \frac{10^n}{a}$

For a given value of $a$ and $p$ we can find the value of $b$ as follows:

$b = \frac{(10^n a)}{(p \cdot a - 10^n)}$

Solutions $a, b, p$ for $n = 1$ are:

$1, 1, 20$

$1, 2, 15$

$1, 5, 12$

$1, 10, 11$

$2, 2, 10$

$2, 5, 7$

$2, 10, 6$

$3, 6, 5$

$3, 15, 4$

$4, 4, 5$

$4, 20, 3$

$5, 5, 4$

$5, 10, 3$

$6, 30, 2$

$10, 10, 2$

$11, 110, 1$

$12, 60, 1$

$14, 35, 1$

$15, 30, 1$

$20, 20, 1$

# Brute Force Approach 

In [ ]:
def solve(n):
    infer = lambda p, a: (10**n * a) / (p * a - 10**n)
    def solve_for_a(a):
        solutions = 0
        p = int(10**n / a) + 1
        b = infer(p, a)
        while b >= a:
            if (abs(int(b) - b) == 0):
                solutions += 1
            p += 1
            b = infer(p, a)
        return solutions
    a = 1
    solutions = 0
    while True:
        solutions += solve_for_a(a)
        a += 1
        p = int(10**n / a) + 1
        b = infer(p, a)
        if b < a:
            break
    return solutions

# Look into prime factorization

In [ ]:
from nmutils.primes import get_prime_factors
def factorize(n):
    factors = get_prime_factors(n)
    if len(factors) > 1:
        return [int(factor) for factor in factors]
    return factors[0] if factors else 1

In [ ]:
solutions = [[1, 1, 20], [1, 2, 15], [1, 5, 12], [1, 10, 11],
             [2, 2, 10], [2, 5, 7], [2, 10, 6], [3, 6, 5],
             [3, 15, 4], [4, 4, 5], [4, 20, 3], [5, 5, 4],
             [5, 10, 3], [6, 30, 2], [10, 10, 2], [11, 110, 1],
             [12, 60, 1], [14, 35, 1], [15, 30, 1], [20, 20, 1]]

In [ ]:
for solution in solutions:
    a, b, p = solution
    print(solution, f"\t-->\t10 x ({a} + {b}) = {p} x {a} x {b}")
    print([factorize(x) for x in solution])
    print('-' * 60)

Fix $a = 3, n = 1$ - then $p_{min} = ceiling(\frac{10^1}{3}) = 4$. If $p = 3$ then $b = \frac{(10^1 \cdot 3)}{(4 \cdot 3 - 10^1)} = \frac{30}{2} = 15$.

So for $a = 3$ and $n = 1$ we have $p \geq 4$ and $b \leq 15$.

Let's look at values of $b$ in the range $[3, 15]$ and see what we get!

In [ ]:
a = 3
for b in range(15, 2, -1):
    print(f"10 x ({a} + {b}) = {10 * (a + b)} = p x {a} x {b} --> p = {10 * (a + b) / (a * b):.3f} --> {10 * (a + b) % (a * b) == 0}")

From this we can see that if $p$ is to divide $10(a+b)$ then the product $a \cdot b$ must divide $10(a+b)$.

In other words, for given $n$ and $a$, we have a solution for $b$ precisely when $10^n(a + b) \mod a \cdot b = 0$!

Does this pose a solution?

For given $n$:
<ol>
    <li>Start with $a=1$</li>
    <li>Calculate $p_{min} = ceiling(\frac{10^n}{a})$</li>
    <li>Calculate $b_{max} = \frac{(10^n a)}{(p_{min} \cdot a - 10^n)}$</li>
    <li>$\forall b \in [a, b_{max}]$, count number of solutions where $10^n(a + b) \mod a \cdot b = 0$</li>
    <li>Increment $a$ and repeat</li>
    <li>Stop when $a \gt b$</li>
</ol>

In [ ]:
from itertools import count, takewhile
from math import ceil

In [ ]:
def elegant(n):
    def solve_for_a(a):
        pmin = ceil(10**n / a)
        if pmin * a == 10**n:
            pmin += 1
        bmax = int((10**n * a) / (pmin * a - 10**n))
        if bmax < a:
            return None
        solutions = 0
        for b in range(a, bmax + 1):
            if 10**n * (a + b) % (a * b) == 0:
                solutions += 1
        return solutions
    infinite_solutions = map(solve_for_a, count(1))
    solutions = takewhile(lambda s: s is not None, infinite_solutions)
    return sum(solutions)

In [ ]:
%%time
[elegant(n) for n in range(1, 4)]

# Combine brute force with elegant solution

In [ ]:
def solve2(n):
    infer = lambda p, a: int((10**n * a) / (p * a - 10**n))
    def solve_for_a(a):
        check_solution = lambda b: 10**n * (a + b) % (a * b) == 0
        solutions = set()#0
        p = int(10**n / a) + 1
        b = infer(p, a)
        while b >= a:
            #if check_solution(b) or check_solution(b + 1):
            if check_solution(b):
                #print(a, b, p)
                solutions.add((a, b))
                #solutions += 1
            p += 1
            b = infer(p, a)
        return len(solutions)
    a = 1
    solutions = 0
    while True:
        solutions += solve_for_a(a)
        a += 1
        p = int(10**n / a) + 1
        b = infer(p, a)
        if b < a:
            break
    print(f"{n}: {solutions}")
    return solutions

In [ ]:
[solve2(n) for n in range(1, 5)]

In [ ]:
20 + 102 + 356 + 958 + 2192 + 4456 + 8230 + 13361 + 18520